# Build a quasar composite spectrum

# About

This notebook creates a composite quasar spectrum from a collection of quasar spectra.
The quasar spectra were retrieved from the *Sloan Digital Sky Survey* database DR 12 (http://skyserver.sdss.org/dr12/en/home.aspx) and stored in FITS files.

The notebook reads all FITS files in the given folder, corrects each individual spectrum for redshift (from observed frame to emitted frame), then re-bin-s the data in bins of a user-defined size. It then continues by calculating normalisation factors for each spectrum by the reciprocal of the division of the average spectral flux density in the overlapping region by the average spectral flux density of the adjacent (in the redshift range) spectrum in the same overlapping region.
Finally, all spectra are combined to one composite spectrum by calculating the *geometric* mean value of all values in a bin, for each bin.

## Credits

This notebook and associated Python modules were developed as part of a group project within the context of module S382 Astrophysics, lectured at the Open University in the UK (http://www.open.ac.uk/courses/modules/s382).

## Initialise

Auto reload libraries, use inline plots

In [ ]:
%load_ext autoreload
#%matplotlib inline

Import the libraries

In [3]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tic
import os

## Create a query for selecting 100 spectra between z = 0.1 and z = 6.9

In [ ]:
num_bins = 100
min_x = 0.1
max_x = 6.9
delta_x = (max_x-min_x)/num_bins

for i in range(1, num_bins):
    left_x = i*delta_x - 0.01
    right_x = i*delta_x + 0.01

    print("SELECT TOP 1 bestObjId, plate, mjd, fiberid")
    print("FROM SpecObj")
    print("WHERE class='QSO'")
    print("AND bestObjId>0")
    print("AND zWarning=0")
    print("AND z BETWEEN {0:.2f} AND {1:.2f}".format(left_x, right_x))
    print("UNION")

i = num_bins
left_x = i*delta_x - 0.01
right_x = i*delta_x + 0.01
print("SELECT TOP 1 bestObjId, plate, mjd, fiberid")
print("FROM SpecObj")
print("WHERE class='QSO'")
print("AND bestObjId>0")
print("AND zWarning=0")
print("AND z BETWEEN {0:.2f} AND {1:.2f}".format(left_x, right_x))

## Query the database

Copy & Paste the generated SQL query in the *Skyserver SQL Search* box at http://skyserver.sdss.org/dr12/en/tools/search/sql.aspx.

Next, download all FITS files - most easily done using the *Bulk Optical Spectra Search* at http://mirror.sdss3.org/bulkSpectra.

## Create a composite spectrum using homebrewn routines

In [4]:
%autoreload
import hkCompositeSpectrum as hsc

#
# Create composite spectrum
#
binSize = 4.
foldername = "./FITS/"

# Store the names of fits-files
filenameList = []
for filename in os.listdir(foldername):
    if filename.endswith(".fits"):
        filenameList.append(foldername + filename)

# Create the composite spectrum
compositeSpectrum = hsc.CombineSpectra(filenameList, binSize = 4., method="GM")

# Store in CSV file
filename = './FITS/hkCompositeSpectrum.csv'
print("Writing composite spectrum to " + filename)
compositeSpectrum.to_csv(filename)
print("Done...")

Processing: ./FITS/spec-6796-56453-0619.fits...
Processing: ./FITS/spec-4046-55605-0120.fits...
Processing: ./FITS/spec-6636-56367-0706.fits...
Processing: ./FITS/spec-4392-55833-0700.fits...
Processing: ./FITS/spec-4266-55486-0028.fits...
Processing: ./FITS/spec-1467-53115-0442.fits...
Processing: ./FITS/spec-5293-55953-0794.fits...
Processing: ./FITS/spec-5732-56326-0378.fits...
Processing: ./FITS/spec-4421-55858-0044.fits...
Processing: ./FITS/spec-3765-55508-0074.fits...
Processing: ./FITS/spec-2366-53741-0470.fits...
Processing: ./FITS/spec-6619-56371-0210.fits...
Processing: ./FITS/spec-1020-52721-0286.fits...
Processing: ./FITS/spec-6421-56274-0206.fits...
Processing: ./FITS/spec-5427-56001-0380.fits...
Processing: ./FITS/spec-5461-56018-0574.fits...
Processing: ./FITS/spec-6121-56187-0167.fits...
Processing: ./FITS/spec-4551-55569-0336.fits...
Processing: ./FITS/spec-1407-52886-0328.fits...
Processing: ./FITS/spec-1375-53084-0424.fits...
Processing: ./FITS/spec-5377-55957-0261.

## Plot the composite spectrum, including common emission lines

In [ ]:
%autoreload
import hkSpectrumLines as hsl

#
# Plot the spectrum
#

# Create a figure with axes
fig = plt.figure()
ax0 = fig.add_subplot(1, 1, 1)

# Plot the composite spectrum
ax0.plot(compositeSpectrum['wavelength'], compositeSpectrum['flux'], linewidth=1)

# Plot spectrum lines
hsl.PlotSpectrumLines(ax0,[r'Ly$\alpha$', 'C IV', 'C III]', 'Mg II', r'H$\beta$', '[O III]', r'H$\alpha$'])

# Set limits for axes
#ax0.set_xlim(1000,8500)
#ax0.set_ylim(0, 1800)

# Create labels for axes
ax0.set_xlabel('Wavelength $\lambda / \AA$', fontsize='large')
ax0.set_ylabel('Spectral flux density $f_{\lambda} \, / $ Arbitrary units', fontsize='large')

# Create a title
ax0.set_title('Spectral flux densities for composite spectrum of quasar', fontsize='xx-large')

# Display a grid
ax0.grid(True)

# Set the tick marks
xMajorLocator = tic.MultipleLocator(500)
xMinorLocator = tic.AutoMinorLocator(5)
ax0.xaxis.set_major_locator(xMajorLocator)
ax0.xaxis.set_minor_locator(xMinorLocator)

yMinorLocator = tic.AutoMinorLocator(5)
ax0.yaxis.set_minor_locator(yMinorLocator)

ax0.tick_params(which = 'both', direction = 'in')

# Display the plot
plt.show()

## Create a log-log plot of the composite spectrum

In [ ]:
#
# Plot the spectrum in loglog scale
#

# Create a figure with axes
fig = plt.figure()
ax0 = fig.add_subplot(1, 1, 1)

# Plot the composite spectrum
ax0.loglog(compositeSpectrum['wavelength'], compositeSpectrum['flux'], linewidth=1)

# Set limits for axes
#ax0.set_xlim(1000,8500)
#ax0.set_ylim(0, 1800)

# Create labels for axes
ax0.set_xlabel('Wavelength $\lambda / \AA$', fontsize='large')
ax0.set_ylabel('Spectral flux density $f_{\lambda} \, / $ Arbitrary units', fontsize='large')

# Create a title
ax0.set_title('Spectral flux densities for composite spectrum of quasar', fontsize='xx-large')

# Display a grid
ax0.grid(True)

# Set the tick marks
xMajorLocator = tic.MultipleLocator(500)
xMinorLocator = tic.AutoMinorLocator(5)
ax0.xaxis.set_major_locator(xMajorLocator)
ax0.xaxis.set_minor_locator(xMinorLocator)

yMinorLocator = tic.AutoMinorLocator(5)
ax0.yaxis.set_minor_locator(yMinorLocator)

ax0.tick_params(which = 'both', direction = 'in')

# Display the plot
plt.show()